In [29]:
import json
from pyspark.sql import Row

# Linux Commands

### cd into project 2 directory
`cd ~/w205/project-2-juliejlai`

### curl down json file into project 2 directory:
`curl -L -o assessment-attempts-20180128-121051-nested.json https://goo.gl/ME6hjp`

### copy yml file:
`cp ~/w205/course-content/08-Querying-Data/docker-compose.yml ~/w205/project-2-juliejlai`

### spin/bring up docker:
`docker-compose up -d`

### check docker:
`docker-compose ps`
`docker ps -a`

### create a mirror console / start kafka logs:
`docker-compose logs -f kafka`

### create a topic (name = assessments):
`docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181`

### check topic:
`docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181`

### write messages using kafka cat:
`docker-compose exec mids bash -c "cat /w205/project-2-juliejlai/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"`

### read the message:
`docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e"`

### spin up pyspark:
`docker-compose exec spark pyspark`

### or bring up jupyter notebook:
`docker-compose exec spark bash`
#### Create a symbolic link from the spark directory to /w205 :
`ln -s /w205 w205`
`exit`
`docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark`

### shut down docker
`docker-compose down`
`docker-compose ps`
`docker ps -a`

# Transforming Messages in Spark

### Unrolling the json (using Kevin's starter code)

In [30]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 
raw_assessments.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [31]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()
extracted_assessments.registerTempTable('assessments')

# Question: How many assessments are in the dataset?

In [32]:
num_assessments = spark.sql("select count(*) as `num_assessments` from assessments")
num_assessments.show()

+---------------+
|num_assessments|
+---------------+
|           3280|
+---------------+



Answer: There are 3280 assessments in the dataset.

# Question: What's the name of your Kafka topic? How did you come up with that name?

Answer: The name of my Kafka topic is 'assessments'. We chose this name in class because it is has signficance to the dataset, which is assessments data.

# Question: How many people took Learning Git?

In [33]:
learn_git = spark.sql("select count(exam_name) as `count_learning_git` from assessments where exam_name = 'Learning Git'")
learn_git.show()

+------------------+
|count_learning_git|
+------------------+
|               394|
+------------------+



Answer: 394 people took Learning Git

# Question: What is the least common course taken? And the most common?


In [34]:
com_course = spark.sql("select exam_name, count(exam_name) as `count` from assessments group by exam_name \
                       having count(exam_name) = (select max(count) from \
                       (select exam_name, count(exam_name) as `count` from assessments group by exam_name)) \
                       or count(exam_name) = (select min(count) from \
                       (select exam_name, count(exam_name) as `count` from assessments group by exam_name))")
com_course.show(truncate = False)

+-------------------------------------------------+-----+
|exam_name                                        |count|
+-------------------------------------------------+-----+
|Learning to Visualize Data with D3.js            |1    |
|Learning Git                                     |394  |
|Nulls, Three-valued Logic and Missing Information|1    |
|Native Web Apps for Android                      |1    |
|Operating Red Hat Enterprise Linux Servers       |1    |
+-------------------------------------------------+-----+



Answer: The least common courses taken are 'Learning to Visualize Data with D3.js', 'Nulls, Three-valued Logic and Missing Information', 'Native Web Apps for Android', 'Operating Red Hat Enterprise Linux Servers' with only 1 person taking each course. The most common course taken is 'Learning Git' with 394 people taking the course.

# Question: On average, how many questions are there per exam? Exams with the lowest number of questions? Exams with the highest number of questions?

### How to handle "holes" in json data (using Kevin's starter code)

In [35]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"exam": raw_dict["exam_name"],
                           "correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

In [36]:
my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()
my_correct_total.registerTempTable('ct')

In [37]:
num_q = spark.sql('select exam, avg(total) as `total` from ct group by exam\
                    having total = (select max(total) from ct) \
                    or total = (select min(total) from ct)')
num_q.show(truncate = False)

+-------------------------------------------------+-----+
|exam                                             |total|
+-------------------------------------------------+-----+
|View Updating                                    |1.0  |
|Introduction to Hadoop YARN                      |1.0  |
|Nullology                                        |1.0  |
|Introduction to Apache Hive                      |1.0  |
|Nulls, Three-valued Logic and Missing Information|1.0  |
|Git Fundamentals for Web Developers              |1.0  |
|Operating Red Hat Enterprise Linux Servers       |20.0 |
+-------------------------------------------------+-----+



In [38]:
avg_q = spark.sql('select avg(total) as `avg_q` from ct')
avg_q.show()

+-----------------+
|            avg_q|
+-----------------+
|4.489160305343511|
+-----------------+



Answer: The average number of questions on an exam is 4.489. The exam with the highest number of questions (20 questions) is 'Operating Red Hat Enterprise Linux Servers'. The exams with the lowest number of questions (1 question) are 'View Updating', 'Introduction to Hadoop YARN', 'Nullology', 'Introduction to Apache Hive', 'Nulls, Three-valued Logic and Missing Information', and 'Git Fundamentals for Web Developers'.

# Question: What are the average score and standard deviation per exam? What are the lowest and highest average scores? Is there a correlation between the number of questions on an exam and the score?

In [39]:
avg_scores = spark.sql("select exam, avg(score)*100 as `avg_score`, std(score) as `std`, avg(total) as `num_q` \
                        from (select exam, (correct / total) as score, total from ct) group by exam order by avg_score desc")

avg_scores.show()

+--------------------+-----------------+-------------------+-----+
|                exam|        avg_score|                std|num_q|
+--------------------+-----------------+-------------------+-----+
|Learning to Visua...|            100.0|                NaN|  3.0|
|Nulls, Three-valu...|            100.0|                NaN|  1.0|
|The Closed World ...|            100.0|                0.0|  2.0|
|Learning SQL for ...|97.72727272727273|0.07537783614444094|  4.0|
|Introduction to J...|87.59493670886073|0.18833172378627616|  5.0|
|Introduction to A...|83.33333333333334| 0.1767766952966369|  4.0|
|Introduction to A...|83.33333333333334|0.27888667551135854|  3.0|
|Cloud Native Arch...|80.00000000000003|0.22677868380553634|  5.0|
|Getting Ready for...|80.00000000000001|                0.2|  5.0|
|Understanding the...|78.57142857142857|0.10101525445522103|  7.0|
|Introduction to A...|76.92307692307693|0.21014172398537173|  3.0|
|Beginning Program...|76.58227848101265|0.23459516463275298|  

Answer: 'Client-Side Data Storage for Web Developers' had the lowest average score with an average score of 20 and standard deviation of around 0.283. The exams with the highest average score were 'Learning to Visualize Data with D3.js', 'Nulls, Three-valued Logic and Missing Information', and 'The Closed World Assumption'. The average scores for these exams were 100. There does not seem to be a correlation between the number of questions on an exam and the score, as there seems to be a similar pattern of variety in the number of questions for both exams with high average scores and exams with low average scores.

# Question: How many unique user_exam_ids exist?

In [40]:
uniq_user = spark.sql("select count(distinct(user_exam_id)) as unique_id from assessments")
uniq_user.show()

+---------+
|unique_id|
+---------+
|     3242|
+---------+



Answer: 3242 unique user_exam_ids exist

# Question: How many exams does each person take?

In [41]:
exam_per_user = spark.sql("select user_exam_id, count(*) as `count` from assessments \
                            group by user_exam_id order by count desc")
exam_per_user.show(truncate = False)

+------------------------------------+-----+
|user_exam_id                        |count|
+------------------------------------+-----+
|1e325cc1-47a9-4808-8f6b-508b5459ed6d|3    |
|b7ac6d15-97e1-4e94-a09d-da819024b8cd|3    |
|a244c11a-d890-4e3e-893d-d17c5ce2ad05|3    |
|3d63ec69-8d97-4f99-82aa-b0786ef21679|3    |
|028ad26f-a89f-4a63-95d4-b6b58f6fa30d|3    |
|fa23b287-0d0a-4683-8d19-38a65b7f57d1|3    |
|00745aef-f3af-4127-855c-afc3b6ef4011|3    |
|cdc5859d-b332-4fb1-aae4-5cacb52cea5f|3    |
|37cf5b0c-4807-4214-8426-fb1731b57700|3    |
|ac80a11a-2e79-40ef-a756-7edb6f0ddf0b|3    |
|949aa36c-74c7-4fc1-a41f-42386c1beb37|3    |
|c320d47f-60d4-49a5-9d6c-67e947979bf0|3    |
|bd96cfbe-1532-4ba2-a504-7e8a437a5065|3    |
|d4ab4aeb-1368-4866-bc5e-7eee69fd1608|3    |
|a7e6fc04-245f-4e3c-9539-e2aac44c0eb8|3    |
|a45b5ee6-a4ed-4b18-b962-15abddd765d7|3    |
|66d91177-c436-4ee1-b0b0-daa960e1b2d0|3    |
|6132da16-2c0c-436c-9c48-43b8bafe0978|3    |
|c1eb4d4a-d6ef-43ee-9ef4-58bc6c1d17ff|2    |
|6e4889ab-

Answer: A person, represented by a unique user_exam_id, has taken a range of 1 to 3 exams.

# Write to HDFS in parquet format

In [42]:
# full dataset
extracted_assessments.write.mode('overwrite').parquet("/tmp/extracted_assessments") 

# number of total assessments
num_assessments.write.mode('overwrite').parquet("/tmp/num_assessments") 

# number of people taking learning git
learn_git.write.mode('overwrite').parquet("/tmp/learn_git") 

# least and most common courses taken
com_course.write.mode('overwrite').parquet("/tmp/com_course") 

# number of correct questions and num of total questions per exam
my_correct_total.write.mode('overwrite').parquet("/tmp/my_correct_total") 

# number of questions per exam
num_q.write.mode('overwrite').parquet("/tmp/num_q") 

# average number of questions for all assessments
avg_q.write.mode('overwrite').parquet("/tmp/avg_q") 

# average score, standard deviation, and number of questions per exam
avg_scores.write.mode('overwrite').parquet("/tmp/avg_scores") 

# number of unique user ids
uniq_user.write.mode('overwrite').parquet("/tmp/uniq_user") 

# number of exams a person has taken
exam_per_user.write.mode('overwrite').parquet("/tmp/exam_per_user") 